# Generate error message retrieve data
## Load LTL tasks from txt file

In [3]:
## These tasks are used to train the embedding model and should be stored in '../GA/tasks.txt'.
file_path = "../GA/tasks.txt"
with open(file_path) as f:
    tasks = f.readlines()
print(tasks[0])
print(len(tasks))

U(O(N(f), E(U(a, b))), N(f))

956


In [ ]:
## Convert the prefix format of these tasks into the original format
def format_convert(task):
    

## Import OpenAI 

In [ ]:
import json, openai
# You need to set your OPENAI API key here
# https://beta.openai.com/account/api-keys
openai.api_key = "TO_BE_SET"

In [ ]:
PROMPT = """Given a linear temporal logic task, specify 10 similar tasks and their expected behaviors, 

Task: 
F(a U (b A Xc))

Outputs:

1.

Possible Mistake: Misplaced parentheses, which result in a different meaning. Revised Task: F((a U b) & Xc)

Possible Mistake: Using until (U) for b and Xc without specifying the order. Revised Task: F((a U b) & (b U Xc))

Possible Mistake: Using until (U) for a and b together, which doesn't specify the requirement of a before b. Revised Task: F(a U b) & Xc

Possible Mistake: Incorrectly using eventually (F) before a until (U) b, which may not capture the desired behavior. Revised Task: F(a U (b & Xc))

Possible Mistake: Using next (X) for b and Xc without specifying their relationship. Revised Task: F(a U (b & Xc))

Possible Mistake: Not including the temporal operator X for c. Revised Task: F(a U (b & X(Xc)))

Possible Mistake: Combining a and b under a single until (U), which may not reflect the intended behavior. Revised Task: F((a & b) U Xc)

Possible Mistake: Using the until (U) operator for both a U b and b U Xc without properly separating them. Revised Task: (F(a U b)) & Xc

Possible Mistake: Applying until (U) to a U b without considering Xc. Revised Task: F(a U b) & Xc

Task:
TASK-TO-BE-PLACED

Outputs:

"""
def rephrase_a_sentence(sentence):
    response = openai.Completion.create(
        model="text-davinci-002",
        prompt=PROMPT.replace("TASK-TO-BE-PLACED", normalize(sentence)),
        temperature=0.7,
        max_tokens=512,
        top_p=1,
        best_of=1,
        frequency_penalty=0.1,
        presence_penalty=0
        )
    output = response['choices'][0]['text']
    try:
        paraphrases = parse_sentences_from_response(output)
    except:
        print("Error in parsing response")
        print(output)
        return output, "ERROR"
    return parse_sentences_from_response(output)